## 1) Generate Sudoku - optimized trial layout

In [ ]:
using SudokuPlantDesign
using BenchmarkTools
using BlockArrays
using DataFrames
using Random
using XLSX
using PyPlot
using ColorTypes
using PyCall
@pyimport matplotlib.colors as matcolors

As explained in `sudoku_basic`, we generate a new configuration `conf` which is divided into 2 horizontal and 5 vertical blocks of dimensions `10` x `2` respectively. In total, there are `4` different checks in the configuration. All fields which are neither checks nor missing plots are defined as entries - these are the unreplicated entries which are augmented by replicated checks. We leave the plot `[1,3]` and `[1,4]` empty. Configuration is plotted again to show the update. Then, checks are initialized randomly with one check per check type per block.

In [ ]:
conf = get_configuration(
    [9,9],  [2,2,2,2],    3
    ;
    bc = :open
);

empty_plots!(conf, 1:1,3:4)
initialize_checks_per_block!(conf)
show_configuration(conf, zoom=0.2)
savefig("output/initial_random_check_distribution.pdf")
sum(conf.configuration .== 0)

We use a standard selection of cost functions, as used in the `sudoku_basic` example.

In [ ]:
function K_indiv(conf :: C) :: Float64 where {C <: CheckConfiguration}
    return  K_num_checks_equal_per_type(conf) +
            K_checks_per_type_per_block(conf, 1)*20+
            K_neighbors_different_check_functional(conf, d->0.5/(d^3)) +
            K_neighbors_same_check_functional(conf, d->1/(d^3))
end

In the Sudoku-augmented design, we use 3 updates since we need to insert new checks, swap checks with each other and swap also checks with entries.

In [ ]:
initialize_entries!(conf, 119)
costs = optimize_design!(
    conf,
    [UpdateNewCheckLabel(),UpdateSwapCheckCheck(),UpdateSwapCheckEntry()],
    K_indiv
)

By inspecting the final plot, we see that the check distribution is optimized.

In [ ]:
costs .-= minimum(costs)
costs .+= 1

#print things
println("costs: ",K_indiv(conf)+costs[1], " -> ", K_indiv(conf)+costs[end])
println("perc_checks: ",conf.num_checks_total / (conf.num_plots_total - conf.num_checks_total))
println("entries: ",conf.num_plots_total - conf.num_checks_total)
println("checks:    ",conf.num_checks_total, "  (", round(100*conf.num_checks_total / conf.num_plots_total, digits=2), "%)")
for i in 1:conf.N
    println("check ($(i)): ",conf.num_checks[i], "  (", round(100*conf.num_checks[i] / conf.num_checks_total, digits=2), "%)")    
end


figure()
plot(costs)
yscale("log")


#save without check labels
show_configuration(conf,check_labels=true)
savefig("output/final_sudoku_design.pdf")

## 6) Plot with Plant_Codes and write data to xlsx

## 2) Save design data with field plan

First, we read in the input data `input_augmented.xlsx` with both sheets `checks` and `entries`. Each sheet contains the name of the entry in the first column, and in following columns further properties of the entries can be added to be transferred to the final output file and field plan. 

In [ ]:
entrydata = string.(DataFrame(XLSX.readtable("input_augmented.xlsx", "entries")));
replace!.(eachcol(entrydata), "missing" => "NA");

In [ ]:
checkdata = string.(DataFrame(XLSX.readtable("input_augmented.xlsx", "checks")));
replace!.(eachcol(checkdata), "missing" => "NA");

Since we usually want the entries to be randomized, their data is internally shuffled.

In [ ]:
lconf = LabeledCheckConfiguration(conf)

fill_indices_snake_y!(lconf, 1,1, index_for_empty=false)
fill_labels!(lconf, checkdata, entrydata)

show_configuration(lconf, check_labels=true)
savefig("output/final_sudoku_design_labels.pdf")

The data table of the optimized Sudoku-augmented design can be saved.

In [ ]:
write_to_xlsx_allinfo(conf.configuration,"Testtrial",POS,L)

To be able to load the design again, we can save the configuration.

In [ ]:
save_configuration(conf, "configuration.txt")